In [1]:
input_file = sc.textFile('input2.txt')

In [2]:
input_file.take(10)

[u"The Project Gutenberg Etext of Plutarch's Lives, by A.H. Clough",
 u'Also known as "Parallel Lives", written in Greek ~100 A.D.',
 u'Includes 50 biographies, 23 Greek, 23 Roman, 2 others.',
 u'',
 u'',
 u'Copyright laws are changing all over the world, be sure to check',
 u'the copyright laws for your country before posting these files!!',
 u'',
 u'Please take a look at the important information in this header.',
 u'We encourage you to keep this file on your own disk, keeping an']

In [3]:
import string
import unicodedata
import re
def delete_punctuation_marks(line):
    punctuation = string.punctuation
    line = re.sub(r'#([^\s]+)',r'\1',line)
    line = re.sub(r'([abdefghijkmnopqstuvwyz])\1+', r'\1', line)
    line = u''.join([i for i in line if not i.isdigit()])
    for punct in punctuation:
        line = re.sub(re.escape(punct),'',line)
    return line.strip().lower().rstrip()


In [4]:
clean_text = input_file.map(delete_punctuation_marks).filter(lambda x: x is not u'')

In [5]:
clean_text.take(20)

[u'the project gutenberg etext of plutarchs lives by ah clough',
 u'also known as parallel lives writen in grek  ad',
 u'includes  biographies  grek  roman  others',
 u'copyright laws are changing all over the world be sure to check',
 u'the copyright laws for your country before posting these files',
 u'please take a lok at the important information in this header',
 u'we encourage you to kep this file on your own disk keping an',
 u'electronic path open for the next readers  do not remove this',
 u'welcome to the world of fre plain vanilla electronic texts',
 u'etexts readable by both humans and by computers since',
 u'these etexts prepared by hundreds of volunters and donations',
 u'information on contacting project gutenberg to get etexts and',
 u'further information is included below  we ned your donations',
 u'plutarchs lives',
 u'edited by ah clough',
 u'october   etext',
 u'the project gutenberg etext of plutarchs lives by ah clough',
 u'this file should be named txt or zip',
 

In [6]:
def count_words_foreach_word(line):
    tuples_list = list()
    line = line.split()
    for i in range(len(line)):
        word = line[i]
        for j in range(len(line)):
            if j==i: continue
            tuples_list.append(((word,line[j]),1))
    return tuples_list

In [7]:
from operator import add

In [8]:
count_text = clean_text.flatMap(count_words_foreach_word).reduceByKey(add).map(lambda x: (x[0][0],(x[0][1],x[1])))

In [9]:
count_text.take(20)

[(u'well', (u'dyers', 1)),
 (u'wield', (u'am', 1)),
 (u'war', (u'mere', 6)),
 (u'lagons', (u'the', 1)),
 (u'castle', (u'prisoners', 1)),
 (u'apearance', (u'god', 1)),
 (u'persians', (u'wives', 1)),
 (u'losed', (u'metaphor', 1)),
 (u'his', (u'banishment', 24)),
 (u'came', (u'trumpets', 1)),
 (u'small', (u'account', 2)),
 (u'about', (u'hung', 4)),
 (u'nations', (u'rhine', 1)),
 (u'proposal', (u'withstod', 1)),
 (u'slow', (u'sure', 1)),
 (u'reckon', (u'hapy', 1)),
 (u'family', (u'objections', 1)),
 (u'eager', (u'expectation', 1)),
 (u'conditions', (u'senate', 1)),
 (u'lifting', (u'with', 4))]

In [10]:
from operator import itemgetter
def top_N(x):
    words_ocurrences = list(x[1])
    words_ocurrences.sort(key=itemgetter(1), reverse = True)
    if len(words_ocurrences) > 10:
        words_ocurrences = words_ocurrences[:10]
    return (x[0], words_ocurrences)

In [11]:
group_count_text = count_text.groupByKey().map(top_N)

In [12]:
group_count_text.take(20)

[(u'fawn',
  [(u'the', 5),
   (u'and', 2),
   (u'decres', 1),
   (u'athenians', 1),
   (u'go', 1),
   (u'let', 1),
   (u'fear', 1),
   (u'hunters', 1),
   (u'from', 1),
   (u'through', 1)]),
 (u'biger',
  [(u'and', 2),
   (u'a', 1),
   (u'but', 1),
   (u'with', 1),
   (u'they', 1),
   (u'away', 1),
   (u'them', 1),
   (u'all', 1),
   (u'chains', 1),
   (u'gylipus', 1)]),
 (u'foul',
  [(u'and', 17),
   (u'the', 16),
   (u'of', 13),
   (u'to', 8),
   (u'fell', 5),
   (u'that', 5),
   (u'upon', 4),
   (u'for', 4),
   (u'or', 4),
   (u'a', 3)]),
 (u'four',
  [(u'and', 94),
   (u'of', 91),
   (u'the', 85),
   (u'a', 37),
   (u'hundred', 36),
   (u'with', 32),
   (u'thousand', 30),
   (u'in', 30),
   (u'to', 28),
   (u'his', 28)]),
 (u'conjuring',
  [(u'the', 4),
   (u'rest', 2),
   (u'them', 2),
   (u'and', 1),
   (u'cried', 1),
   (u'anaphlystian', 1),
   (u'by', 1),
   (u'of', 1),
   (u'guardian', 1),
   (u'army', 1)]),
 (u'cyprus',
  [(u'the', 25),
   (u'of', 23),
   (u'and', 21),
   (u'

In [13]:
all_words = group_count_text.collect()

In [14]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [15]:
with open("first-part_output.txt", 'w') as output:
    for a in all_words:
        list_words = ' '.join(str(e).encode('utf-8').strip() for e in a[1])
        output.write(a[0]+'\t'+list_words+'\n')

In [61]:
list_words

"oracular [(u'or', 1), (u'an', 1), (u'designing', 1), (u'pleasure', 1), (u'own', 1), (u'his', 1), (u'to', 1), (u'whether', 1), (u'advice', 1), (u'erect', 1)]"